In [1]:
import helpers
from helpers import *

### W2

In [2]:
qreg = QuantumRegister(2)
w4 = QuantumCircuit(qreg)
w4.x(qreg[0])
Bdirect(w4, qreg, 1, 0, 0.5)
print(w4)
job = qiskit.execute(w4, Aer.get_backend('statevector_simulator'))
theorical_psi = job.result().get_statevector(w4)
print(theorical_psi)

                ┌───┐                              ┌───┐
q0_0: |0>───────┤ X ├─────────■────────────────────┤ X ├
         ┌──────┴───┴──────┐┌─┴─┐┌────────────────┐└─┬─┘
q0_1: |0>┤ U3(-0.7854,0,0) ├┤ X ├┤ U3(0.7854,0,0) ├──■──
         └─────────────────┘└───┘└────────────────┘     
[ 0.        +0.j -0.70710678+0.j  0.70710678+0.j  0.        +0.j]


In [3]:
tomo_circuits = state_tomography_circuits(w4, qreg)

In [4]:
job = qiskit.execute(tomo_circuits, melbourne, shots=4000)

calib_circuit, state_labels = mc.complete_meas_cal(qr=qreg)
job_cal = qiskit.execute(calib_circuit, melbourne, shots=4000)

job_monitor(job, monitor_async = True)

job_monitor(job_cal, monitor_async = True)

HTML(value="<p style='font-size:16px;'>Job Status: job is queued </p>")

HTML(value="<p style='font-size:16px;'>Job Status: job is being initialized </p>")

In [5]:
cal_results = job_cal.result()
meas_fitter = mc.CompleteMeasFitter(cal_results, state_labels)
correct_results = meas_fitter.filter.apply(job.result())
fitter = StateTomographyFitter(correct_results, tomo_circuits)
fitted_rho = fitter.fit()
fidelity = state_fidelity(theorical_psi, fitted_rho)
print(fidelity)

0.9464407051043953


In [6]:
# 0.958, 0.946